# CodeBert Grid Experiment Evaluation

Nice to see you around! Have a seat.
Would you like a drink? Maybe a cigar?

In [ ]:
import os

In [ ]:
data_directory = "./data/PreliminaryResults"

print(f"looking for results in {data_directory}" )

property_files={}
result_files={}

for root,dirs,files in os.walk(data_directory):
    for name in files:
        if "reference.gold" in name:
            result_files["gold"]=os.path.join(root,name)
        elif "reference.output" in name:
            result_files["reference"]=os.path.join(root,name)
        elif "config_" in root and ".output" in name:
            result_files[os.path.basename(root)]=os.path.join(root,name)
        elif "config_" in root and ".properties" in name:
            property_files[os.path.basename(root)]=os.path.join(root,name)

print(f"There were {len(result_files)} result- and {len(property_files)} property-files found.")
# Sanity Checks 
if "gold" not in result_files.keys():
    print("There was no Gold-file for the results found!")
if "reference" not in result_files.keys():
    print("There was no reference-file for the results found!")

In [ ]:
# initialize bare results
results = {}
# Fill them with file-paths
for (key,value) in result_files.items():
    results[key] = {}
    results[key]["result_file"] = value

for (key,value) in property_files.items():
    results[key]["property_file"] = value

In [ ]:
def load_properties(filepath, sep='=', comment_char='#'):
    """
    Read the file passed as parameter as a properties file.
    """
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(comment_char):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = sep.join(key_value[1:]).strip().strip('"') 
                props[key] = value 
    return props


In [ ]:
print("reading in property-files")

for key in property_files.keys():
    prop_file = results[key]["property_file"]
    if prop_file:
        results[f"{key}"]["properties"]=load_properties(prop_file)

In [ ]:
print("reading in result-files")

for key in results.keys():
    result_file = results[key]["result_file"]
    if result_file:
        f = open(result_file)
        lines=f.readlines()
        results[key]["results"]={}
        for l in lines:
            num = int(l.split("\t")[0])
            content = l.split("\t")[1]
            content = content.strip()
            results[key]["results"][num] = content
        f.close()

In [ ]:
results

In [ ]:
import bleu_evaluator as foreign_bleu

In [ ]:
foreign_bleu.bleuFromMaps(results["gold"]["results"],results["reference"]["results"])

In [ ]:
foreign_bleu.bleuFromMaps(results["gold"]["results"],results["config_0"]["results"])

In [ ]:
%%time
for key in results.keys():
    if "gold" not in key:
        bleu = foreign_bleu.bleuFromMaps(results["gold"]["results"],results[key]["results"])
        results[key]["bleu"]=bleu